### Define the Neural Network to work with 

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel(gray) 6 output channels, 5x5 square filter
        #kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an Affine operation : y=Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self, x):
        #Max pool over a 2x2 window
        x=F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        #If the size is a square, specify a single number
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size=x.size()[1:] # all dims except the first (batch)
        num_features=1
        for s in size:
            num_features *= s
        return num_features
net=Net()
print(net)
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### View weights

In [30]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's weight

10
torch.Size([6, 1, 5, 5])


In [31]:
# Try a random 32x32 input
input=torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.1093, -0.0861,  0.1855,  0.0112,  0.0197, -0.0103,  0.0845,  0.2008,
         -0.0885,  0.0803]], grad_fn=<ThAddmmBackward>)


In [32]:
net.zero_grad()
out.backward(torch.randn(1,10))

### Calculate Loss

In [33]:
output = net(input)
target =torch.randn(10) # dummy target
target = target.view(1,-1)
criterion = nn.MSELoss()

loss=criterion(output,target)
print(loss)

tensor(0.9964, grad_fn=<MseLossBackward>)


In [34]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])

print(loss.grad_fn.next_functions[0][0].next_functions[0][0])


### Backpropogate Gradients into weights

In [35]:
#net.zero_grad() # zeros the gradient buffers on all parameters but already done before

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('after backward')
print(net.conv1.bias.grad)



conv1.bias.grad before backward
tensor([ 0.0899,  0.1049,  0.2010, -0.0855,  0.0699,  0.1866])
after backward
tensor([ 0.0811,  0.0592,  0.1812, -0.0910,  0.0691,  0.1681])


In [36]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)
# in training loop
optimizer.zero_grad()
output=net(input)
loss=criterion(output, target)
loss.backward()
optimizer.step() #Executes the updating